# 📸 Computational Photography

**Topics:** HDR, Tone Mapping, Image Blending

In [ ]:
# Setup
!pip install opencv-python-headless numpy matplotlib -q
import cv2
import numpy as np
import matplotlib.pyplot as plt
print('✅ Setup complete!')

In [ ]:
# HDR Tone Mapping Simulation
# Create synthetic HDR image
np.random.seed(42)
hdr = np.random.rand(200, 300).astype(np.float32) * 10  # High dynamic range

# Different tone mapping operators
def reinhard_tonemap(hdr, key=0.18):
    """Reinhard global operator"""
    lum = hdr
    lum_avg = np.exp(np.mean(np.log(lum + 1e-6)))
    lum_scaled = (key / lum_avg) * lum
    return lum_scaled / (1 + lum_scaled)

def gamma_tonemap(hdr, gamma=2.2):
    """Simple gamma curve"""
    return np.clip(hdr, 0, 1) ** (1/gamma)

# Apply tone mapping
ldr_reinhard = reinhard_tonemap(hdr)
ldr_gamma = gamma_tonemap(hdr / hdr.max())
ldr_clip = np.clip(hdr / hdr.max(), 0, 1)

fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes[0,0].imshow(np.log1p(hdr), cmap='gray')
axes[0,0].set_title('HDR (log scale)')
axes[0,1].imshow(ldr_clip, cmap='gray')
axes[0,1].set_title('Simple Clip')
axes[1,0].imshow(ldr_reinhard, cmap='gray')
axes[1,0].set_title('Reinhard Tone Map')
axes[1,1].imshow(ldr_gamma, cmap='gray')
axes[1,1].set_title('Gamma Correction')
for ax in axes.flat: ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Image Blending with Pyramid
def build_gaussian_pyramid(img, levels):
    pyr = [img]
    for _ in range(levels):
        img = cv2.pyrDown(img)
        pyr.append(img)
    return pyr

def build_laplacian_pyramid(img, levels):
    gauss_pyr = build_gaussian_pyramid(img, levels)
    lap_pyr = []
    for i in range(levels):
        expanded = cv2.pyrUp(gauss_pyr[i+1], dstsize=(gauss_pyr[i].shape[1], gauss_pyr[i].shape[0]))
        lap_pyr.append(gauss_pyr[i].astype(float) - expanded.astype(float))
    lap_pyr.append(gauss_pyr[-1].astype(float))
    return lap_pyr

# Create two images
img1 = np.zeros((256, 256), dtype=np.uint8)
img1[:, :128] = 255
img2 = np.zeros((256, 256), dtype=np.uint8)
cv2.circle(img2, (128, 128), 80, 255, -1)

# Build pyramids
levels = 4
lap1 = build_laplacian_pyramid(img1, levels)
lap2 = build_laplacian_pyramid(img2, levels)

# Create mask pyramid
mask = np.zeros((256, 256), dtype=np.float32)
mask[:, :128] = 1
mask_pyr = build_gaussian_pyramid(mask, levels)

# Blend pyramids
blended_pyr = []
for l1, l2, m in zip(lap1, lap2, mask_pyr):
    m = m[:l1.shape[0], :l1.shape[1]]
    blended_pyr.append(l1 * m + l2 * (1 - m))

# Reconstruct
result = blended_pyr[-1]
for i in range(levels - 1, -1, -1):
    result = cv2.pyrUp(result, dstsize=(blended_pyr[i].shape[1], blended_pyr[i].shape[0]))
    result = result + blended_pyr[i]

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(img1, cmap='gray')
axes[0].set_title('Image 1')
axes[1].imshow(img2, cmap='gray')
axes[1].set_title('Image 2')
axes[2].imshow(np.clip(result, 0, 255).astype(np.uint8), cmap='gray')
axes[2].set_title('Pyramid Blending')
plt.show()